`comparaison_env`

In [ ]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *

Variables globales

In [ ]:
### MODELNET40 REMESHING ISO
ModelNet40_aligned_us = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R22.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print("Fichiers bvs de Modelnet40 :", len(paths_bvs))
dir_bvs = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/bvs_remeshing_iso"

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
dir_us = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude.csv")
label_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_rep_etude_label.csv")
data_us_cam['label'] = list(label_us_cam['label'])
## Choix et bvs des 44 modeles 
dir_bvs_us = '/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific/csv_etude_filtre/visualisation_filtre'
paths_bvs_us_csv = glob.glob(os.path.join(dir_bvs_us, "**", "*global_distribution_label.csv*"), recursive=True); print("Modeles de US : ",len(paths_bvs_us_csv))
# Path data folder of user study
dir_Data = "/home/pelissier/These-ATER/Papier_internationale2/Data"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'aeroplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet', 'night_stand': 'cabinet', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}
# Outputs tmp
# Path of user-study outputs folder in Dataset
dir_outputs = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/user_study"

## Contraintes : quelles caméras de lUS on considère ?

In [ ]:
## Condition
## Toutes les caméras 
#j_cam_us_ok = [0,1,2,3,4]

## Que les cameras de la couronne j==1
j_cam_us_ok = [1,2]

## Que les cameras de la couronne j==1 ou j == 2
#j_cam_us_ok = [1,2]

## Cameras

In [ ]:
## User study
# Coordonnées des caméras de l'étude utilateur
X_us = []; Y_us = []; Z_us = []; labels_us = []; I_us = []; J_us = []
for j in j_cam_us_ok:
    X_us += list(data_us_cam.loc[data_us_cam['j'] == j]['X_rep_etude'])
    Y_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Y_rep_etude'])
    Z_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Z_rep_etude'])
    I_us += list(data_us_cam.loc[data_us_cam['j'] == j]['i'])
    J_us += [j for _ in range(8)]
    labels_us += list(data_us_cam.loc[data_us_cam['j'] == j]['label']) 
cams_us = np.around(np.column_stack((X_us, Y_us, Z_us, np.array([1]*len(X_us)))),3)
R_sphere = list(data_us_cam['R'])[0]
## ModeleNet 
# Coordonnées des 12 caméras de ModelNet40 dans le repère de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])
cams_modelnet = np.column_stack((X_modelnet, Y_modelnet, Z_modelnet, np.array([1]*12)))
print("12 cams Modelnet dans repère Modelnet : \n", cams_modelnet)

In [ ]:
print(len(cams_us), len(labels_us))
print(str(len(cams_us))+" cams US considérées dans repère US : \n", cams_us, labels_us)

## BVS user study --> en fonction des caméras de l'us

In [ ]:
all_bvs_us = {}
# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_bvs_us_csv in paths_bvs_us_csv:
    name = os.path.basename(path_bvs_us_csv).split('_')[0]
    df = pd.read_csv(path_bvs_us_csv)
    filter_df = df[df['label'].isin(labels_us)]
    filter_sorted_df = filter_df.sort_values(by='poids', ascending=False)
    # BVS : attention il put y avoir plusieurs BVS pour un même modèle <=> plusieurs labels avec le même poids
    label_bvs_mesh = list(filter_sorted_df.loc[filter_sorted_df['poids'] ==  max(list(filter_sorted_df['poids']))]['label'])
    position_bvs_mesh = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_mesh]]
    # coordonnées 3D des caméras BVS dans le repère de l'US
    cam_bvs = np.concatenate([np.array(X_us)[position_bvs_mesh], np.array(Y_us)[position_bvs_mesh], np.array(Z_us)[position_bvs_mesh]]).T  
    # Indice I-J des caméras dans le repère de l'US
    i_j_cam = np.concatenate([np.array(I_us)[position_bvs_mesh], np.array(J_us)[position_bvs_mesh]]).T
    all_bvs_us[name] = {'df': filter_sorted_df, 'label_bvs': label_bvs_mesh, 'cam_bvs': cam_bvs, 'ij_bvs': i_j_cam}

In [ ]:
all_bvs_us['carVasa']

## Pour 1 categorie

In [ ]:
categorie_modelnet = 'car'
categorie_us = match_ModelNet2US[categorie_modelnet]; print(categorie_modelnet, categorie_us)
## Fichiers BVS de la categorie
paths_bvs = [paths_bvs[i] for i in range(len(paths_bvs)) if categorie_modelnet in paths_bvs[i]]; print("Fichiers bvs de la categorie dispo :", len(paths_bvs), paths_bvs[0])
random.shuffle(paths_bvs)

### Mise en correspondance cameras : Pour 1 objet 3D

In [ ]:
# A partir du mesh aligné à l'US, on applique les memes transformations au 12 caméras de ModelNet40
# Pour les places dans le repère US (=cams_modelnet_mesh)
# Puis on détermine les coordonnées de la cam BVS dans le repère US (=cam_bvs_model et num_cam_bvs_model)
def bvs_cams_modelnet_aligned(path_mesh, path_mesh_modelnet_aligned, dir_bvs, cams_modelnet):
    # Transformation du mesh de ModelNet40 en mesh de l'étude utilisateur
    transformations = read_pkl(path_mesh_modelnet_aligned.replace(".obj", ".pkl"))
    # Transformation des 12 cameras pour mettre dans le repère de l'étude utilisateur comme le modèle 
    cams_modelnet_mesh = cams_modelnet.copy()
    for n in range(0, len(transformations)):
        # Rotation Rn
        R = transformations[f"transformations{n}"]
        cams_modelnet_mesh = np.dot(R, cams_modelnet_mesh.T).T
        
    ## BVS ATTENTION DANS LE CALCULS DE LA BVS ON A BOUCLE SUR RANGE(1,13) ET NON PAS SUR RANGE(0,12)
    num_cam_bvs_modelnet = read_pkl(os.path.join(dir_bvs, os.path.basename(path_mesh)+"_bvs.pkl"))['bvs'].split('_')[-1]
    cam_bvs_modelnet = cams_modelnet_mesh[int(num_cam_bvs_modelnet)-1][:3]    
    return cams_modelnet_mesh, cam_bvs_modelnet, num_cam_bvs_modelnet

In [ ]:
# cameras_modelnet : cameras BVS de ModelNet40 DANS LE REPERE US !!!!
# cameras_US : caméras de l'US considérées dans le repère US
def toto(df, camera_modelnet, centroid_modelnet, cameras_US, sigma = 0.01, precision = 8):
    # Projection de la caméra BVS de ModelNet40 sur la sphère des caméras de l'US
    camera_modelnet_sphere = put_cam_on_sphere(R_sphere, camera_modelnet, centroid_modelnet); print("Camera modelnet sur la sphere : ", camera_modelnet_sphere)
    poids = []
    for k in range(len(cameras_US)):
        # poids de cam_sphere vis à vis de cam_etude_k
        poids_k = np.round(gaussian(sigma, cameras_US[k, :3], camera_modelnet_sphere), precision)  ;print("Poids de la caméra modelnet sur la caméra US ", int(I_us[k]), J_us[k], ": ", poids_k)
        poids.append(poids_k)
    # Add the list poids as the 16 last columns of df and at the last row
    for k in range(len(poids)):
        df.loc[len(df)-1, f"{int(I_us[k])}-{J_us[k]}"] = poids[k]
    return df, camera_modelnet_sphere

In [ ]:
df_poids_from_modelnet = pd.DataFrame(columns=['path', 'name']+[f"{int(I_us[k])}-{J_us[k]}" for k in range(len(I_us))])
# Pour chacun des modèles de la catégorie étudée dont on a le fichier bvs (que 38 actuellement), issue de ModelNet40 (car : 296)
for num in tqdm.tqdm(range(len(paths_bvs)*0+1)):
    ###################################################################
    ## ModelNet40
    # Load model alignés à l'étude utilisateur
    path_mesh = 'car/train/car_0089_SMPLER_centered_scaled_remeshing_iso_iter5' #paths_bvs[num]; print(path_mesh)
    name_modelnet =  '_'.join(os.path.basename(path_mesh).split('_')[:2])
    df_poids_from_modelnet.loc[0, 'path'] = path_mesh; df_poids_from_modelnet.loc[0, 'name'] = name_modelnet
    # Load mesh PRÉALABLEMENT placé dans le REPRÈRE US (avec code Alignenment/align_mesh.ipynb)
    path_mesh_modelnet_aligned = os.path.join(ModelNet40_aligned_us, path_mesh+"_aligned_ok_US.obj")
    mesh_modelnet_aligned = trimesh.load_mesh(path_mesh_modelnet_aligned)
    centroid_modelnet_aligned = get_centroid(mesh_modelnet_aligned.faces, mesh_modelnet_aligned.vertices)
    # BVS du mesh ModelNet40 aligned
    cams_modelnet_mesh, cam_bvs_modelnet, num_cam_bvs_modelnet = bvs_cams_modelnet_aligned(path_mesh, path_mesh_modelnet_aligned, dir_bvs, cams_modelnet)
    print(f'Modelnet : cam_{num_cam_bvs_modelnet}', cam_bvs_modelnet)
    
    ## Objectif : trouver la "BVS moyenne"  <--> attribué un poids d'impact a chacun des cam de l'US
    # Impact de la camera BVS de modlenet40 sur les caméras de l'étude utilisateur
    df_a, cam_sphere = toto(df_poids_from_modelnet, cam_bvs_modelnet, centroid_modelnet_aligned, cams_us, sigma = 0.58, precision = 3)

    
    ###################################################################
    ## User stuy
    # mesh random from User_study
    path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj"); #print(path_mesh_us)
    mesh_us = trimesh.load_mesh(path_mesh_us)
    # BVS 
    cam_bvs_us = all_bvs_us[categorie_us]['cam_bvs']; print('US : ', cam_bvs_us, all_bvs_us[categorie_us]['label_bvs'], all_bvs_us[categorie_us]['ij_bvs'])

    ## Superposition : US et modelent40
    #show_cams(mesh_modelnet, cams_modelnet_mesh, name_modelnet, mesh_us, cams_us, categorie_us,  dir_outputs)
    
    ###################################################################
    ## Score de proximité 
    #score = score_proximite(cam_bvs_modelnet_in_us, cam_bvs_us)
    
    

In [ ]:
gaussian(0.58, [0,0,0], [0,0,0])

In [ ]:
show_cams(mesh_modelnet_aligned, np.array([cam_sphere, cam_bvs_modelnet]), "toto", mesh_us, cams_us, categorie_us,  dir_outputs)

# Debug

In [ ]:
a

In [ ]:
# ## OBJ file with (12 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_modelnet = []#np.array(mesh_modelnet.vertices)
# faces_mesh_modelnet = []#np.array(mesh_modelnet.faces) 
# # colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
# colormap = plt.get_cmap('hot'); colors_modelnet = colormap(np.linspace(0, 1, 12))[::-1] 
# #with open(os.path.join(dir_outputs, categorie_modelnet+"_modelNet40+12cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_modelnet:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 12 cameras positions
#     vertex_offset = len(verts_mesh_modelnet)
#     for cube_center, cube_color in zip(cams_modelnet_mesh[:, :3], colors_modelnet[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_modelnet:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
        
# ################################################################################"" 
# ## OBJ file with (8 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_us = np.array(mesh_us.vertices)
# faces_mesh_us = np.array(mesh_us.faces) 
# # colors : Jaune cam 1 --> Vert cam 3 ---> Bleu cam 5 --> Bleu foncé cam 8
# colormap = plt.get_cmap('hot'); colors_us = colormap(np.linspace(0, 1, 8))[::-1] 
# # Write obj
# #with open(os.path.join(dir_outputs, categorie_us+"_us+8cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_us:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 8 cameras positions
#     vertex_offset = len(verts_mesh_us)
#     for cube_center, cube_color in zip(cams_us[:, :3], colors_us[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_us:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   

### BVS : match celle de l'US et celle de ModelNet40